In [43]:
# !pip install newsdataapi

In [44]:
import openai
import json
import os 
import numpy as np
from newsdataapi import NewsDataApiClient
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
import credentials

In [45]:
def get_news(searchquery:str=None)->list:
    '''
    input searchquery is a string that takes in a variable that needs to be searched. It is an optional variable
    The function returns a list of string of latest news
    '''
    import random
    api = NewsDataApiClient(apikey=credentials.NEWSDATA_KEY)
    
    page=None
    news= []
    # while True:
    #     response = api.news_api(page = page,q= searchquery , country = "us")
    #     page = response.get('nextPage',None)
    #     if not page:
    #         break
    #     else:
    #         news.append(page)
    if searchquery:
        response = api.news_api(q= searchquery,country = "in", language = 'en')
    else:
        response = api.news_api(country = "in", language = 'en')
    
    
    news = [content['content'] for content in response['results']]
    # news_dict = {i:content['content'] for i,content in enumerate(response['results'])}

        
    # {"1":"asda","2":"asd"}        
    return random.choice(news)
    

In [46]:
api = NewsDataApiClient(apikey=credentials.NEWSDATA_KEY)

In [47]:
newssummary = Tool(
    name='newssummary',
    func = get_news,
    description='get latest news as a list of string'
)

In [48]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo-0613',openai_api_key=credentials.OPENAI_KEY)

In [49]:
tools = [newssummary]
agent = initialize_agent(tools,llm,agent=AgentType.OPENAI_FUNCTIONS,verbose = True)

In [50]:
import langchain
langchain.debug = False

In [52]:
from time import sleep

limit = "175 characters"
responses = []
prompt = f"""
Your task is to get the latest news and then generate a short summary for this news \
Summarize the news 
in at most {limit}. Try to be as short as possible. Never ever cross the {limit} limit for summarized news.
"""

for i in range(5):
    sleep(30)
    response = agent.run(prompt)
    responses.append(response)

for response in responses:
    print(len(response.split(" ")))



> Entering new  chain...

Invoking: `newssummary` with `{'source': 'latest'}`


Preliminary investigation has revealed that the accused carried out recces. Seven people have been arrested in connection with the gunpoint robbery of a delivery agent and his associate inside the Pragati Maidan tunnel here, police said on Tuesday. The accused robbed them of about Rs 2 lakh on Saturday when they were on their way in a taxi to Gurugram to deliver the money. Preliminary investigation has revealed that the accused carried out recces on Thursday and Friday, and were on the look out. On Saturday, they executed the robbery, a senior police officer said. CCTV camera footage received by the Delhi Police showed four motorcycle-borne men intercepted the car and robbed the delivery agent of Omiya Enterprises, Chandni Chowk, and his associate. The arrested are Usman, his cousin Irfan, Anuj Mishra, Kuldeep, Sumit, Pradeep and Bala, police said. They were following the taxi and chose to stop it inside 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-sTxGXte3ziCUC03xQshXnu9v on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-sTxGXte3ziCUC03xQshXnu9v on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues..


Congress and DMK question PM Modi's pitch for a uniform civil code. DMK argues it should be applied to Hindus first, allowing all castes to pray in temples. Congress criticizes Modi for not addressing poverty and unemployment. PM Modi defends uniform civil code, questions why Triple Talaq isn't practiced in Muslim-majority countries. Uniform civil code refers to overarching laws that replace religion-based personal laws. Law commission seeks views on the issue.

> Finished chain.
1
1
1
1
1


In [58]:
len(responses[0].split(" "))

54

In [55]:
responses

['Seven people have been arrested for the gunpoint robbery of a delivery agent and his associate in Delhi. The accused robbed them of Rs 2 lakh while they were on their way to deliver the money. CCTV footage showed four men on motorcycles intercepting the car inside a tunnel and fleeing with the money.',
 "AAP demands CAG probe into Kejriwal's bungalow renovation. The Union Home Ministry has ordered an audit following allegations of irregularities. AAP accuses BJP of targeting political opponents. CAG to conduct special audit into financial irregularities. AAP urges investigation into scams in BJP-ruled states.",
 "Pune attack: Man with machete chases and attacks woman in street. Locals overpower him and hand him over to police. Attacker arrested and charged with attempted murder. Victim's mother says he had been harassing her daughter.",
 "Congress and DMK question PM Modi's pitch for a uniform civil code. DMK argues it should be applied to Hindus first, allowing all castes to pray in